In [1]:
import requests
import pandas as pd
from IPython.core.debugger import set_trace

In [28]:
class NotProperResponseError(Exception):
    pass

class NotProperInputError(Exception):
    pass

class SomethingWrongError(Exception):
    pass


def _int(num_s):
    try:
        return int(num_s)
    except ValueError:
        return int(float(num_s))

In [67]:
def request_hist_hour(nbars=None, utc_to=None, coin=None, base_ccy=None, ex=None):    
    # 참고: 종료시점(utc_to)이 반드시 시간(h)으로 나누어 떨어질 필요는 없다
    
    # 모든 파라미터가 제공되어야 한다
    if (nbars is None) or (utc_to is None) or (coin is None) or (base_ccy is None) or (ex is None):
        raise NotProperInputError()
    
    # 종료시점(utc_to)이 utc timestamp(integer/float 또는 숫자로 된 문자열)로 주어진 경우
    if str(utc_to).isdigit():
        utc_to = _int(utc_to)
        
    # 시간 문자열(ex. 2019-06-12 05:00:45)로 주어진 경우
    else:
        utc_to = int(pd.Timestamp(utc_to).timestamp())
        
        
    baseurl = ('https://min-api.cryptocompare.com/data/histohour?'
               'fsym={fsym}'
               '&tsym={tsym}'
               '&e={e}'
               '&limit={limit}'
               '&toTs={toTs}')
    
    url = baseurl.format(fsym=coin, tsym=base_ccy, e=ex, limit=nbars, toTs=utc_to)
    resp = requests.get(url).json()
    msg = resp['Response']
    
    if msg == 'Success':
        truncated = True
        utc_from = None
        
        data = {str(pd.to_datetime(_r.pop('time'), unit='s')):_r for _r in resp['Data'] if _r['close'] != 0}
        
        if len(data) > 0:
            n_del = max(0, len(data) - nbars)
            keys = sorted(data)
            keys_del = keys[:n_del]
            [data.pop(k) for k in keys_del]
        
            utc_from = keys[n_del]
        
            if len(data) == nbars:
                truncated = False
                
        
        return {
            'data': data, 
            'truncated': truncated, 
            'utc_from': utc_from, 
        }
    
    elif msg == 'Error':
        raise NotProperResponseError()
        
    else:
        raise SomethingWrongError()

In [30]:
req = request_hist_hour(nbars=3, utc_to='2018-12-26 08:30', coin='EOS', base_ccy='KRW', ex='bithumb'); req

{'data': {'2018-12-26 06:00:00': {'close': 2890,
   'high': 2926,
   'low': 2875,
   'open': 2915,
   'volumefrom': 163020.31,
   'volumeto': 473246392.31},
  '2018-12-26 07:00:00': {'close': 2874,
   'high': 2899,
   'low': 2839,
   'open': 2890,
   'volumefrom': 290628.29,
   'volumeto': 831545786.19},
  '2018-12-26 08:00:00': {'close': 2850,
   'high': 2881,
   'low': 2811,
   'open': 2874,
   'volumefrom': 209549.29,
   'volumeto': 595417744.44}},
 'truncated': False,
 'utc_from': '2018-12-26 06:00:00'}

In [68]:
def request_hist_hour_range(utc_from=None, utc_to=None, coin=None, base_ccy=None, ex=None):
    # 시작시점(utc_from)이나 종료시점(utc_to)이 주어지지 않았다면, 
    # 해당 코인 가격정보를 가장 최근 시점까지 가져온다
    # 참고: 입력 파라미터에서의 시작시점(utc_from)와 종료시점(utc_to)이 반드시 시간(h)으로 나누어 떨어질 필요는 없다
    
    data = {}
    
    if utc_from is None:
        utc_from = pd.Timestamp('2000-01-01')
    else:
        utc_from = pd.Timestamp(utc_from).floor('h')
        
    if utc_to is None:
        utc_to = pd.Timestamp.utcnow().floor('h') + pd.Timedelta(-1, unit='h')
    else:
        utc_to = pd.Timestamp(utc_to).floor('h')
    
    
    # nbar 구하기: 시작시점(_from)을 포함해야 하기에 +1
    _nbars_h = lambda _from, _to: int((_to.timestamp() - _from.timestamp()) / 3600) + 1        
        
    while True:
        nbars = _nbars_h(utc_from, utc_to)
        nbars = min(2000, nbars)
        req = request_hist_hour(nbars=nbars, utc_to=utc_to, coin=coin, base_ccy=base_ccy, ex=ex)
        data.update(req['data'])
        
        print(req['utc_from'] + ' - ' + str(utc_to))
        
        if req['truncated'] or (nbars < 2000): 
            print('completed')
            break
        
        # 루프를 계속 돌려야된다면, 전 response의 (시작시점-1초)를 다음 루프의 종료시점으로 둔다
        else:
            utc_to = pd.Timestamp(req['utc_from']) + pd.Timedelta(-1, unit='s')
            
            
    return data

In [69]:
rs = request_hist_hour_range(coin='BTC', base_ccy='KRW', ex='upbit')

2019-04-05 00:00:00 - 2019-06-27 07:00:00+00:00
2019-01-11 16:00:00 - 2019-04-04 23:59:59
2018-10-20 08:00:00 - 2019-01-11 15:59:59
2018-07-29 00:00:00 - 2018-10-20 07:59:59
2018-05-06 16:00:00 - 2018-07-28 23:59:59
2018-04-27 22:00:00 - 2018-05-06 15:59:59
completed


In [53]:
pd.DataFrame(rs).T.sort_index()

,close,high,low,open,volumefrom,volumeto
2018-04-27 22:00:00,9929000.0,9989000.0,9928000.0,9955000.0,150.61,1.500340e+09
2018-04-27 23:00:00,9831000.0,9961000.0,9778000.0,9929000.0,256.25,2.533206e+09
2018-04-28 00:00:00,9887000.0,9961000.0,9752000.0,9831000.0,384.32,3.797137e+09
2018-04-28 01:00:00,9984000.0,9987000.0,9877000.0,9887000.0,373.28,3.710482e+09
2018-04-28 02:00:00,9929000.0,9987000.0,9928000.0,9984000.0,297.95,2.969332e+09
2018-04-28 03:00:00,9985000.0,10000000.0,9922000.0,9929000.0,280.62,2.795841e+09
2018-04-28 04:00:00,9995000.0,10016000.0,9969000.0,9985000.0,440.40,4.401185e+09
2018-04-28 05:00:00,9982000.0,10007000.0,9972000.0,9995000.0,300.46,3.000933e+09
2018-04-28 06:00:00,9939000.0,9990000.0,9926000.0,9982000.0,416.14,4.147045e+09
2018-04-28 07:00:00,9920000.0,9957000.0,9900000.0,9939000.0,353.12,3.505581e+09


In [96]:
# url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=1'
# url = 'https://api.coingecko.com/api/v3/exchanges/bithumb'
url = 'https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=KRW&limit=1&e=upbit'
resp = requests.get(url).json(); resp

{'Response': 'Success',
 'Type': 100,
 'Aggregated': False,
 'Data': [{'time': 1561525200,
   'close': 14951000,
   'high': 15592000,
   'low': 14930000,
   'open': 14939000,
   'volumefrom': 494.6,
   'volumeto': 7513933948.41},
  {'time': 1561528800,
   'close': 15119000,
   'high': 15153000,
   'low': 14951000,
   'open': 14951000,
   'volumefrom': 192.89,
   'volumeto': 2904418221.13}],
 'TimeTo': 1561528800,
 'TimeFrom': 1561525200,
 'FirstValueInArray': True,
 'ConversionType': {'type': 'force_direct', 'conversionSymbol': ''},
 'RateLimit': {},
 'HasWarning': False}

In [75]:
prices = pd.DataFrame(resp['Data'])
prices

,close,high,low,open,time,volumefrom,volumeto
0,14286000,14410000,14178000,14304000,1561510800,367.01,5.249753e+09
1,14300000,14307000,14043000,14286000,1561514400,428.64,6.082490e+09
2,14697000,14699000,14279000,14300000,1561518000,423.25,6.105994e+09
3,14842000,14842000,14571000,14697000,1561521600,321.21,4.710727e+09


In [7]:
pd.Timestamp('2019-06-01 01:00:00')

Timestamp('2019-06-01 01:00:00')

In [11]:
pd.to_datetime('1561510800.05', unit='s')

Timestamp('2019-06-26 01:00:00.049999952')